In [1]:
import torch
import pandas as pd
import pyaging as pya
import os

In [2]:
os.system("curl -o coefficients.csv https://www.aging-us.com/article/101508/supplementary/SD5/0/aging-v10i7-101508-supplementary-material-SD5.csv")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 54842  100 54842    0     0   142k      0 --:--:-- --:--:-- --:--:--  142k


0

In [3]:
df = pd.read_csv('coefficients.csv')

df['feature'] = df['ID']
df['coefficient'] = df['Coef']

In [4]:
features = df['feature'][1:].tolist()

weights = torch.tensor(df['coefficient'][1:].tolist()).unsqueeze(0)
intercept = torch.tensor([df['coefficient'][0]])

In [5]:
model = pya.models.LinearModel(len(features))

model.linear.weight.data = weights
model.linear.bias.data = intercept

model

LinearModel(
  (linear): Linear(in_features=391, out_features=1, bias=True)
)

In [6]:
weights_dict = {
    'preprocessing': None, 
    'preprocessing_helper': None,
    'postprocessing': 'anti_log_linear',
    'postprocessing_helper': None,
    'features': features,
    'weight_dict': model.state_dict(),
}

metadata_dict = {
    'species': 'Homo sapiens',
    'data_type': 'methylation',
    'year': 2018,
    'citation': "Horvath, Steve, et al. \"Epigenetic clock for skin and blood cells applied to Hutchinson Gilford Progeria Syndrome and ex vivo studies.\" Aging (Albany NY) 10.7 (2018): 1758.",
    'doi': "https://doi.org/10.18632/aging.101508",
    "notes": None,
}

In [7]:
torch.save(weights_dict, '../weights/skinandblood.pt')
torch.save(metadata_dict, '../metadata/skinandblood.pt')

In [8]:
os.system("rm coefficients.csv")

0